In [ ]:
def compile_chrom_pos_heatmap_from_SVs(SV_df, chrom):
    
    # Get all SVs on 'chrom'
    temp_chrom_df = SV_df[SV_df['CHROM'] == chrom]
    
    # Get the size of the array needed (at max_position of chromosome) 
    temp_max_pos = np.max(temp_chrom_df['END'])
    
    ## Note the first position is 0 here, and will never have a value > 0
    # Create array of 0's based on max_position
    heatmap_array = np.zeros(temp_max_pos + 1)
    
    # Add 1 to array based on position of deletion
    for index, row in temp_chrom_df.iterrows():
        
        heatmap_array[row['POS']:(row['END']+1)] += 1
    
    return heatmap_array

In [ ]:
def ranges(i):
    
    for a, b in itertools.groupby(enumerate(i), lambda pair: pair[1] - pair[0]):
    
        b = list(b)
        
        yield b[0][1], b[-1][1]

In [ ]:
def chrom_df_to_interval_df(chrom_df):
    
    output_df = pd.DataFrame(columns = ['chr', 'overlap_count', 'start', 'end'])
    
    for index, row in chrom_df.iterrows():
        
        temp_range_list = list(ranges(ast.literal_eval(row['positions'])))
        
        for interval in temp_range_list:
            
            temp_row = {'chr': row['chrom'], 'overlap_count': row['overlap_count'], \
                       'start': interval[0], 'end': interval[1]}
            
            output_df = output_df.append(temp_row, ignore_index=True)
            
    return output_df

In [ ]:
def find_max_list(list_of_names):
    
    list_len = [len(globals()[i]) for i in list_of_names]
    
    return max(list_len)

In [ ]:
def percent_overlap(input_start, input_end, gene_start, gene_end):
    
    o  = min(input_end, gene_end) - max(input_start, gene_start) + 1    
    perc = np.round(o/float(int(gene_end)-int(gene_start)+1)*100, 2)
    
    return perc

In [ ]:
def overlap_func(input_chr, input_start, input_end, input_ind, input_sv_type, input_sample_id, df):
    
    output_df = df[df['chr'] == input_chr]
    output_df = output_df[(output_df['end'] >= input_start) & (output_df['start'] <= input_end)]
        
    if len(output_df) != 0:
        
        output_df['percent_overlap'] = output_df.apply(lambda row: \
                                                 percent_overlap(input_start, input_end, row['start'], row['end']), axis=1)

        output_df['input_start'] = input_start
        output_df['input_end'] = input_end
        output_df['input_ind'] = input_ind
        output_df['input_sv_type'] = input_sv_type
        output_df['input_sample_id'] = input_sample_id
        
    
    return output_df

In [ ]:
def overlap_func_wo_typeandid(input_chr, input_start, input_end, input_ind, df):
    
    output_df = df[df['chr'] == input_chr]
    output_df = output_df[(output_df['end'] >= input_start) & (output_df['start'] <= input_end)]
        
    if len(output_df) != 0:
        
        output_df['percent_overlap'] = output_df.apply(lambda row: \
                                                 percent_overlap(input_start, input_end, row['start'], row['end']), axis=1)

        output_df['input_start'] = input_start
        output_df['input_end'] = input_end
        output_df['input_ind'] = input_ind
        
    
    return output_df